# 模範解答: カメラ画像の操作と基本画像処理

以下は課題の模範解答です。

In [ ]:
import cv2
import numpy as np

# スライドバー用コールバック関数
def nothing(x):
    pass

# カメラ初期化
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("カメラが見つかりませんでした")
    exit()

# ズーム倍率の初期値
zoom = 1.0
zoom_step = 0.1  # ズームの増減ステップ

# スライドバーの作成
cv2.namedWindow('Blue Extraction')
cv2.createTrackbar('Blue Min', 'Blue Extraction', 0, 255, nothing)
cv2.createTrackbar('Blue Max', 'Blue Extraction', 255, 255, nothing)

while True:
    ret, frame = cap.read()
    if not ret:
        print("フレームの取得に失敗しました")
        break
    
    # ズーム処理
    h, w, _ = frame.shape
    center_x, center_y = w // 2, h // 2
    new_w, new_h = int(w / zoom), int(h / zoom)
    cropped_frame = frame[center_y-new_h//2:center_y+new_h//2, center_x-new_w//2:center_x+new_w//2]
    frame_zoomed = cv2.resize(cropped_frame, (w, h))
    
    # 大津の二値化
    gray = cv2.cvtColor(frame_zoomed, cv2.COLOR_BGR2GRAY)
    _, otsu_thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 青色の抽出
    blue_min = cv2.getTrackbarPos('Blue Min', 'Blue Extraction')
    blue_max = cv2.getTrackbarPos('Blue Max', 'Blue Extraction')
    blue_channel = frame_zoomed[:, :, 0]
    mask = cv2.inRange(blue_channel, blue_min, blue_max)
    blue_extracted = cv2.bitwise_and(frame_zoomed, frame_zoomed, mask=mask)
    
    # 結果を表示
    cv2.imshow('Zoomed Frame', frame_zoomed)
    cv2.imshow('Otsu Threshold', otsu_thresh)
    cv2.imshow('Blue Extraction', blue_extracted)
    
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # Escキーで終了
        break
    elif key == ord('+'):  # '+'キーでズームイン
        zoom = min(zoom + zoom_step, 3.0)
    elif key == ord('-'):  # '-'キーでズームアウト
        zoom = max(zoom - zoom_step, 1.0)

cap.release()
cv2.destroyAllWindows()